# tf.Variable vs tf.get_variable

https://stackoverflow.com/questions/37098546/difference-between-variable-and-get-variable-in-tensorflow

I'd recommend to always use tf.get_variable(...) -- it will make it way easier to refactor your code if you need to share variables at any time, e.g. in a multi-gpu setting (see the multi-gpu CIFAR example). There is no downside to it.

# tf.Variable always create a new variable

In [6]:
import tensorflow as tf
tf.reset_default_graph()

a = tf.Variable(1., name="v", expected_shape=[1]) 
b = tf.Variable(1., name="v", expected_shape=[1])

init = tf.global_variables_initializer()

assert(a is not b)  # Assertion is true; they are different objects
print(a)
print(b)

with tf.Session() as sess:
    init.run()
    print(sess.run([a, b]))

<tf.Variable 'v:0' shape=() dtype=float32_ref>
<tf.Variable 'v_1:0' shape=() dtype=float32_ref>
[1.0, 1.0]


# tf.get_variable either shares or creates a new one.

- Case 1: If there is no variable with given name (including prefix) used yet, it creates a new one.

- Case 2: If there is a variable with given name (including prefix) used before and if we declare to reuse, it uses or shares the old one. 

- Case 3: If there is a variable with given name (including prefix) used before but if we don't declare to reuse, it raises ValueError. 

In [7]:
import tensorflow as tf
tf.reset_default_graph()

a = tf.get_variable("v", shape=[1]) 
b = tf.get_variable("v", shape=[1])

# ValueError: Variable v already exists, disallowed. 
# Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope? Originally defined at:

ValueError: Variable v already exists, disallowed. Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope? Originally defined at:

  File "/Users/sungchullee/anaconda3/lib/python3.6/site-packages/tensorflow_core/python/framework/ops.py", line 1748, in __init__
    self._traceback = tf_stack.extract_stack()
  File "/Users/sungchullee/anaconda3/lib/python3.6/site-packages/tensorflow_core/python/framework/ops.py", line 3426, in _create_op_internal
    op_def=op_def)
  File "/Users/sungchullee/anaconda3/lib/python3.6/site-packages/tensorflow_core/python/framework/ops.py", line 3357, in create_op
    attrs, op_def, compute_device)
  File "/Users/sungchullee/anaconda3/lib/python3.6/site-packages/tensorflow_core/python/util/deprecation.py", line 507, in new_func
    return func(*args, **kwargs)
  File "/Users/sungchullee/anaconda3/lib/python3.6/site-packages/tensorflow_core/python/framework/op_def_library.py", line 794, in _apply_op_helper
    op_def=op_def)


In [8]:
import tensorflow as tf
tf.reset_default_graph()

a = tf.get_variable("v", shape=[1]) 
b = tf.get_variable("v", shape=[1], reuse=True)

# TypeError: get_variable() got an unexpected keyword argument 'reuse'

TypeError: get_variable() got an unexpected keyword argument 'reuse'

In [9]:
import tensorflow as tf
tf.reset_default_graph()

with tf.variable_scope("one"):
    a = tf.get_variable("v", shape=[1]) 
with tf.variable_scope("one"):
    b = tf.get_variable("v", shape=[1]) 

# ValueError: Variable v already exists, disallowed. 
# Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope? Originally defined at:

ValueError: Variable one/v already exists, disallowed. Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope? Originally defined at:

  File "/Users/sungchullee/anaconda3/lib/python3.6/site-packages/tensorflow_core/python/framework/ops.py", line 1748, in __init__
    self._traceback = tf_stack.extract_stack()
  File "/Users/sungchullee/anaconda3/lib/python3.6/site-packages/tensorflow_core/python/framework/ops.py", line 3426, in _create_op_internal
    op_def=op_def)
  File "/Users/sungchullee/anaconda3/lib/python3.6/site-packages/tensorflow_core/python/framework/ops.py", line 3357, in create_op
    attrs, op_def, compute_device)
  File "/Users/sungchullee/anaconda3/lib/python3.6/site-packages/tensorflow_core/python/util/deprecation.py", line 507, in new_func
    return func(*args, **kwargs)
  File "/Users/sungchullee/anaconda3/lib/python3.6/site-packages/tensorflow_core/python/framework/op_def_library.py", line 794, in _apply_op_helper
    op_def=op_def)


# Sharing Variables

In [3]:
import tensorflow as tf
tf.reset_default_graph()

with tf.variable_scope("one"):
    a = tf.get_variable("v", shape=[1]) 
with tf.variable_scope("one", reuse=True):
    b = tf.get_variable("v", shape=[1]) 
    
init = tf.global_variables_initializer()
    
assert(a is b)  # Assertion is true; they are same objects
print(a)
print(b)

with tf.Session() as sess:
    init.run()
    print(sess.run([a, b]))

<tf.Variable 'one/v:0' shape=(1,) dtype=float32_ref>
<tf.Variable 'one/v:0' shape=(1,) dtype=float32_ref>
[array([-0.42356515], dtype=float32), array([-0.42356515], dtype=float32)]


In [4]:
import tensorflow as tf
tf.reset_default_graph()

with tf.variable_scope("one"):
    a = tf.get_variable("v", shape=[1]) 
    tf.get_variable_scope().reuse_variables()
    b = tf.get_variable("v", shape=[1]) 
    
init = tf.global_variables_initializer()
    
assert(a is b)  # Assertion is true; they are same objects
print(a)
print(b)

with tf.Session() as sess:
    init.run()
    print(sess.run([a, b]))

<tf.Variable 'one/v:0' shape=(1,) dtype=float32_ref>
<tf.Variable 'one/v:0' shape=(1,) dtype=float32_ref>
[array([-0.5155046], dtype=float32), array([-0.5155046], dtype=float32)]


In [5]:
import tensorflow as tf
tf.reset_default_graph()

with tf.variable_scope("one", reuse=tf.AUTO_REUSE):
    a = tf.get_variable("v", shape=[1]) 
    b = tf.get_variable("v", shape=[1]) 
    
init = tf.global_variables_initializer()
    
assert(a is b)  # Assertion is true; they are same objects
print(a)
print(b)

with tf.Session() as sess:
    init.run()
    print(sess.run([a, b]))

<tf.Variable 'one/v:0' shape=(1,) dtype=float32_ref>
<tf.Variable 'one/v:0' shape=(1,) dtype=float32_ref>
[array([0.15242434], dtype=float32), array([0.15242434], dtype=float32)]
